In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from os.path import join
folder = join('..', 'data', 'laws')

In [15]:
import fastText
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
import sklearn
import pymorphy2
import tqdm
import random
from sklearn.model_selection import train_test_split
morph = pymorphy2.MorphAnalyzer()

In [3]:
texts = []
for i in range(1,26):
    texts += open(join(folder, '{}.txt'.format(i)), encoding='utf-8')

In [32]:
# очищает текст от знаков преминания
# возвращает массив из слов текста в нормальной форме

def process_text(text):
    s = "".join(text)
    s = re.sub(r"[^A-Za-zА-Яа-я]+", " ", s)
    words = [word.lower() for word in s.split(" ")]
    
    words_ = []
    for word in tqdm.tqdm_notebook(words):
        words_ += [morph.parse(word)[0].normal_form]
    return words_

In [5]:
#words = process_text(texts)

In [6]:
#s = " ".join(words)
#tmp = open('tmp.txt', mode='w')
#tmp.write(s)

In [7]:
model = fastText.train_unsupervised('tmp.txt')

In [8]:
words_ = open('tmp.txt', encoding='utf-8').readline()
words_ = words_.split(" ")
words_ = np.unique(words_)
words_ = words_[1:]

In [9]:
X = [model.get_word_vector(word) for word in words_]

In [10]:
word2X = dict(zip(words_, X))

In [11]:
kNN = NearestNeighbors(n_neighbors=20).fit(X)

In [16]:
# нахождение 'синонимов' той же части речи

def find_syn(word):
    w = morph.parse(word)[0].normal_form
    try:
        ind = kNN.kneighbors([word2X[w]])[1]
    except KeyError:
        print("Данного слова нет в словаре")
        return
    for w_ in words_[ind].ravel():
        if morph.parse(w)[0].tag.cyr_repr.split(',')[0] == morph.parse(w_)[0].tag.cyr_repr.split(',')[0]:
            print(w_)

In [17]:
find_syn('утверждать')

утверждать
утверждаться
утвердить
раскрывать
дорабатывать
устанавливать
устанавливаться
избегать
выселять
согласовывать


**Классификация глаголов**

In [18]:
verbs = []

for word in words_:
    if morph.parse(word)[0].tag.cyr_repr.split(',')[0] == "ИНФ":
        verbs.append(word)
        print(word)

абортировать
адаптировать
адресовать
аккредитовать
аккумулировать
амортизировать
амортизироваться
анализировать
аннулировать
аннулироваться
арендовать
аттестовать
аффилировать
бальзамировать
бидти
блокировать
браковать
бросить
буксировать
быть
варить
ввезти
ввести
вводить
вводиться
ввозить
ведать
вернуть
вернуться
вести
вестись
взаимодействовать
взвесить
взимать
взиматься
взыскать
взыскивать
взыскиваться
взять
видеть
вкладывать
включать
включаться
включить
владеть
влечь
влиять
вложить
вменить
вменяться
вмешиваться
вмещать
внедрять
внести
вносить
вноситься
вовлекать
возбудить
возбуждать
возвести
возводить
возвратить
возвращать
возвращаться
возглавлять
возглавляться
возлагать
возлагаться
возложить
возместить
возмещать
возмещаться
возникать
возникнуть
возобновить
возобновлять
возобновляться
возражать
войти
воспитывать
воспользоваться
воспрепятствовать
воспрещаться
воспринимать
восстанавливать
восстанавливаться
восстановить
востребовать
вручать
вручаться
вручить
вскрыть
встать
встречать
вс

разведать
развивать
развиваться
разграничить
разделить
разделять
разделяться
разместить
размещать
размещаться
разобрать
разрабатывать
разрабатываться
разработать
разрешать
разрешаться
разрешить
разрушать
разубоживать
разъяснить
разъяснять
разъясняться
районировать
раскрывать
раскрыть
располагать
располагаться
расположить
распоряжаться
распределить
распределять
распределяться
распространить
распространять
распространяться
рассматривать
рассматриваться
рассмотреть
рассрочить
рассчитать
рассчитывать
рассчитываться
растворить
расторгаться
расторгнуть
расходовать
расходоваться
реабилитировать
реализовать
реализоваться
реализовывать
реализовываться
регистрировать
регистрироваться
регламентировать
регламентироваться
регулировать
регулироваться
резервировать
рекомендовать
реконструировать
рекуперировать
реорганизовать
реорганизоваться
реструктурировать
решать
решаться
руководить
руководствоваться
самортизировать
санкционировать
свидетельствовать
связать
связывать
сгруппировать
сдавать
сдаватьс

In [19]:
dict_words = dict(zip(verbs, np.zeros(len(verbs), dtype=int)))

In [20]:
with open('verbs.txt', 'r') as f:
    for verb in f.readlines():
        dict_words[verb[:-1]] = 1

In [21]:
dict_words

{'прилагаться': 0,
 'исправляться': 0,
 'транспортировать': 0,
 'взыскать': 0,
 'пиленгаснуть': 0,
 'описать': 0,
 'ликвидироваться': 0,
 'проживать': 0,
 'влиять': 0,
 'рассчитать': 1,
 'завоевать': 0,
 'специализировать': 0,
 'опубликовывать': 0,
 'подлежать': 0,
 'зафиксировать': 0,
 'разделить': 0,
 'возобновить': 1,
 'заменяться': 0,
 'удержать': 0,
 'издавать': 1,
 'резервировать': 0,
 'отгрузить': 0,
 'плавать': 0,
 'заблудиться': 0,
 'подчиняться': 0,
 'проголосовать': 0,
 'персонифицировать': 0,
 'выкупить': 0,
 'удостоверяться': 0,
 'поручить': 0,
 'закончить': 0,
 'подписывать': 1,
 'группировать': 0,
 'разрабатываться': 0,
 'фиксироваться': 0,
 'опубликовать': 0,
 'вести': 0,
 'запланировать': 0,
 'отгружаться': 0,
 'реорганизовать': 0,
 'переустроить': 0,
 'ограничить': 1,
 'назвать': 0,
 'использовать': 1,
 'распределять': 1,
 'задерживать': 0,
 'вноситься': 0,
 'перечислить': 1,
 'финансироваться': 0,
 'защитить': 1,
 'коксоваться': 0,
 'поделить': 0,
 'оценить': 1,
 'ле

In [22]:
X = [model.get_word_vector(word) for word in verbs]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, list(dict_words.values()), test_size=0.3)

In [24]:
from sklearn.linear_model import LogisticRegression


In [25]:
lr = LogisticRegression(C=1000)
lr.fit(X_train, y_train)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([743, 139]))

In [27]:
y_pred = lr.predict(X_test)

In [28]:
from sklearn.metrics import *

In [29]:
accuracy_score(y_test, y_pred)

0.8095238095238095

In [30]:
test_text = open(join(folder, 'test.txt'), encoding='utf-8').readlines()

In [33]:
words_ = process_text(test_text)

HBox(children=(IntProgress(value=0, max=22068), HTML(value='')))

In [34]:
verbs = []

for word in words_:
    if morph.parse(word)[0].tag.cyr_repr.split(',')[0] == "ИНФ":
        verbs.append(word)
        print(word)

принять
одобрить
иметь
обязать
сохранять
относиться
являться
проживать
определять
обеспечивать
регулировать
возникать
составлять
являться
использоваться
обеспечивать
составлять
сохранить
изменить
создать
обладать
иметь
создать
обладать
иметь
взаимодействовать
связать
объединить
подвергнуться
характеризоваться
сформировать
направить
характеризоваться
обеспечивать
приводить
использоваться
быть
использовать
иметь
оказывать
загрязнять
превышать
оказывать
обеспечиваться
сохраняться
установить
обеспечиваться
установить
соблюдаться
установить
обеспечиваться
сохраняться
загрязнять
определяться
объесть
загрязнять
определяться
объесть
установить
привести
установить
обеспечиваться
загрязнять
загрязнять
планировать
происходить
утратить
направить
предъявлять
определять
повлечь
иметь
вызвать
подпадать
включить
разрушать
озоноразрушать
определяться
озоноразрушать
озоноразрушать
озоноразрушать
озоноразрушать
озоноразрушать
содержаться
озоноразрушать
рекуперировать
озоноразрушать
озоноразрушать
озонора

обосновать
облучить
осуществляться
облучить
устанавливаться
исходить
принимать
возвратить
образоваться
обеспечить
обязать
выполнять
использовать
принимать
запрещаться
подвергаться
запрещаться
создать
осуществлять
запрещаться
изменить
являться
соблюдаться
осуществлять
обязать
обеспечивать
разрабатывать
осуществлять
подлежать
быть
регулироваться
запрещаться
прилегать
быть
создать
использовать
утратить
озоноразрушать
дать
регулироваться
утратить
охранять
устанавливаться
оказывать
создаваться
регулироваться
обеспечиваться
устанавливаться
озоноразрушать
подлежать
объесть
озоноразрушать
вводиться
осуществлять
озоноразрушать
озоноразрушать
подлежать
объесть
озоноразрушать
осуществлять
озоноразрушать
устанавливаться
обязать
соблюдать
обязать
принимать
звереть
разрабатываться
обеспечивать
запрещаться
осуществлять
быть
ограничить
приостановить
прекратить
устанавливаться
устанавливаться
находиться
иметь
находиться
устанавливаться
создаваться
охранять
охранять
регулироваться
охранять
охранять
имет

утрачивать
привести


In [35]:
verbs = np.unique(verbs)

In [36]:
test = [model.get_word_vector(word) for word in verbs]

In [37]:
pred = lr.predict(test)

In [38]:
for i in range(len(pred)):
    if pred[i] == 1:
        print(verbs[i])

быть
взаимодействовать
выдать
дать
допускаться
направить
нести
отнести
перерабатывать
прекратить
признать
приостановить
расположить
рекомендовать
стать
сформировать
учреждаться
